In [2]:
from selenium import webdriver   
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import os
import pandas as pd

Choose the expiration time by using
Dec 2016, Jan 2017 and Feb 2017.

In [114]:
stocks = ['GOOG','YHOO','AAPL','MSFT','AMZN','JPM','BAC','New','CIT','GS']

In [115]:
#12.16; 1.20, 2.17; 
dates = ['1481846400','1484870400','1487289600']

## Example

In [21]:
url = 'http://finance.yahoo.com/quote/GOOG/options?straddle=true&date=1481846400'
chromedriver = "/Users/Miya/Downloads/chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get(url)
html_source = driver.page_source 
table = soup.findAll('table','straddles')
table_html = str(table)
HTML(table_html)

Last Price,Change,% Change,Volume,Open Interest,Strike,Last Price,Change,% Change,Volume,Open Interest
302.80,0.00,0.00%,1,4,440.00,0.20,0.00,0.00%,2,6
323.00,30.10,10.28%,1,4,450.00,0.10,0.00,0.00%,1,77
261.00,0.00,0.00%,4,6,460.00,0.65,0.00,0.00%,1,1
251.10,0.00,0.00%,2,7,470.00,1.00,0.00,0.00%,1,23
292.80,0.00,0.00%,1,35,480.00,0.31,0.00,0.00%,7,115
-,-,-,-,-,490.00,1.05,0.00,0.00%,1,22
280.10,0.00,0.00%,1,9,500.00,0.20,0.00,0.00%,25,106
270.90,0.00,0.00%,1,4,520.00,0.25,-0.25,-50.00%,1,62
248.77,0.00,0.00%,1,9,540.00,0.59,0.00,0.00%,50,120
211.70,0.00,0.00%,1,46,560.00,0.50,-0.20,-28.57%,1,81


In [116]:
pwd

"C:\\Users\\Miya\\OneDrive\\Miya'sGithub\\Financial-Modeling"

In [117]:
mkdir data-quiz5

In [118]:
cd data-quiz5

C:\Users\Miya\OneDrive\Miya'sGithub\Financial-Modeling\data-quiz5


## Crawl All Data

**Option Data on Yahoo Finance is dynamic.(Javascript) Therefore, small tricks are needed when BeautifulSoup is used.**

In [ ]:
chromedriver = "/Users/Miya/Downloads/chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
df_dict = {}
for stock in stocks:
    for date in dates:
        ## Get the url
        url = 'http://finance.yahoo.com/quote/'+ stock +'/options?straddle=true&date='+ date
        ## Crawl data
        driver.get(url)
        html_source = driver.page_source 
        ## Beautifulsoup
        soup = BeautifulSoup(html_source,'html.parser')  
        
        if not soup.find('table','straddles') is None:
            continue
            
        title = [i.text for i in soup.find('table','straddles').find_all('th')]
        text = [i.text for i in soup.find('table','straddles').find_all('td')]
        rows = [row for row in soup.find('table','straddles').find_all("tr")]

        l_table = len(rows) - 1
        ## call data 
        dictionary_call = {}
        dictionary_call['type'] =  ['Call']*l_table
        dictionary_call['option'] = [stock]*l_table
        dictionary_call['Date'] = [date]*l_table
        
        for j in range(6):
            key = title[j]
            dictionary_call[key] = []
            for i in range(l_table):
                dictionary_call[key].append(text[11*i+j])
                
                
        ## put data        
        dictionary_put = {}
        dictionary_put['type'] =  ['Put']*l_table
        dictionary_put['option'] = [stock]*l_table
        dictionary_put['Date'] = [date]*l_table
        
        for j in range(6,11):
            key = title[j]
            dictionary_put[key] = []
            for i in range(l_table):
                dictionary_put[key].append(text[11*i+j])
                
        ## write into dataframe
        df_put = pd.DataFrame(dictionary_put)
        df_call = pd.DataFrame(dictionary_call)
        
        
        df = pd.concat([df_call,df_put])
        
        df_dict[stock+date] = df
        
        df.to_csv(stock+date+'.csv')

In [ ]:
dictionary_call = {}
for j in range(6):
    key = title[j]
    dictionary_call[key] = []
    for i in range(73):
        dictionary_call[key].append(text[11*i+j])

In [88]:
len(dictionary_call['Volume'])

73

In [56]:
rows = [row for row in soup.find('table','straddles').find_all("tr")]
len(rows)

74

In [62]:
text

['302.80',
 '0.00',
 '0.00%',
 '1',
 '4',
 '440.00',
 '0.20',
 '0.00',
 '0.00%',
 '2',
 '6',
 '323.00',
 '30.10',
 '10.28%',
 '1',
 '4',
 '450.00',
 '0.10',
 '0.00',
 '0.00%',
 '1',
 '77',
 '261.00',
 '0.00',
 '0.00%',
 '4',
 '6',
 '460.00',
 '0.65',
 '0.00',
 '0.00%',
 '1',
 '1',
 '251.10',
 '0.00',
 '0.00%',
 '2',
 '7',
 '470.00',
 '1.00',
 '0.00',
 '0.00%',
 '1',
 '23',
 '292.80',
 '0.00',
 '0.00%',
 '1',
 '35',
 '480.00',
 '0.31',
 '0.00',
 '0.00%',
 '7',
 '115',
 '-',
 '-',
 '-',
 '-',
 '-',
 '490.00',
 '1.05',
 '0.00',
 '0.00%',
 '1',
 '22',
 '280.10',
 '0.00',
 '0.00%',
 '1',
 '9',
 '500.00',
 '0.20',
 '0.00',
 '0.00%',
 '25',
 '106',
 '270.90',
 '0.00',
 '0.00%',
 '1',
 '4',
 '520.00',
 '0.25',
 '-0.25',
 '-50.00%',
 '1',
 '62',
 '248.77',
 '0.00',
 '0.00%',
 '1',
 '9',
 '540.00',
 '0.59',
 '0.00',
 '0.00%',
 '50',
 '120',
 '211.70',
 '0.00',
 '0.00%',
 '1',
 '46',
 '560.00',
 '0.50',
 '-0.20',
 '-28.57%',
 '1',
 '81',
 '191.20',
 '2.40',
 '1.27%',
 '1',
 '104',
 '580.00',
 '0.

In [59]:
(74-1)*11 == len(text)

True

In [42]:
text[0]

'302.80'

In [43]:
text[1]

'0.00'

In [54]:
803/11

73.0